# Python con Numba

###### URL fuente: https://pybonacci.org/2015/03/13/como-acelerar-tu-codigo-python-con-numba/

In [1]:
pip install numba

Note: you may need to restart the kernel to use updated packages.


In [3]:
 
import numpy as np
import numba
np.random.seed(0)

In [4]:
data = np.random.randn(2000, 2000)

In [5]:
 
def busca_min(malla):
    minimosx = []
    minimosy = []
    for i in range(1, malla.shape[1]-1):
        for j in range(1, malla.shape[0]-1):
            if (malla[j, i] < malla[j-1, i-1] and
                malla[j, i] < malla[j-1, i] and
                malla[j, i] < malla[j-1, i+1] and
                malla[j, i] < malla[j, i-1] and
                malla[j, i] < malla[j, i+1] and
                malla[j, i] < malla[j+1, i-1] and
                malla[j, i] < malla[j+1, i] and
                malla[j, i] < malla[j+1, i+1]):
                minimosx.append(i)
                minimosy.append(j)
    return np.array(minimosx), np.array(minimosy)

In [6]:

busca_min(data)

(array([   1,    1,    1, ..., 1998, 1998, 1998]),
 array([   1,    3,   11, ..., 1968, 1977, 1985]))

In [7]:
 
mx, my = busca_min(data)
mx.size / data.size

0.11091025

In [8]:
mx.size

443641

In [9]:
def busca_min_np(malla):
    minimos = np.zeros_like(malla, dtype=bool)
    for i in range(1, malla.shape[1]-1):
        for j in range(1, malla.shape[0]-1):
            if (malla[j, i] < malla[j-1, i-1] and
                malla[j, i] < malla[j-1, i] and
                malla[j, i] < malla[j-1, i+1] and
                malla[j, i] < malla[j, i-1] and
                malla[j, i] < malla[j, i+1] and
                malla[j, i] < malla[j+1, i-1] and
                malla[j, i] < malla[j+1, i] and
                malla[j, i] < malla[j+1, i+1]):
                minimos[i, j] = True
    return np.nonzero(minimos)

In [10]:
np.testing.assert_array_equal(busca_min(data)[0], busca_min_np(data)[0])
np.testing.assert_array_equal(busca_min(data)[1], busca_min_np(data)[1])

# Rendimiento

In [11]:

%timeit busca_min(data)

5.14 s ± 81.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit busca_min_np(data)

4.93 s ± 45.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Numba

In [13]:
busca_min_jit = numba.jit(nopython=True)(busca_min)
busca_min_jit(data)

(array([   1,    1,    1, ..., 1998, 1998, 1998], dtype=int64),
 array([   1,    3,   11, ..., 1968, 1977, 1985], dtype=int64))

In [14]:
busca_min_jit_object = numba.jit()(busca_min)

In [15]:
%timeit busca_min_jit_object(data)

88.4 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
busca_min_np_jit = numba.jit(nopython=True)(busca_min_np)
busca_min_np_jit(data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of Function(<function zeros_like at 0x0000017E49A37708>) with argument(s) of type(s): (array(float64, 2d, C), dtype=Function(<class 'bool'>))
 * parameterized
[1mIn definition 0:[0m
[1m    All templates rejected with literals.[0m
[1mIn definition 1:[0m
[1m    All templates rejected without literals.[0m
[1mThis error is usually caused by passing an argument of a type that is unsupported by the named function.[0m[0m
[0m[1m[1] During: resolving callee type: Function(<function zeros_like at 0x0000017E49A37708>)[0m
[0m[1m[2] During: typing of call at <ipython-input-9-11538cc9808c> (2)
[0m
[1m
File "<ipython-input-9-11538cc9808c>", line 2:[0m
[1mdef busca_min_np(malla):
[1m    minimos = np.zeros_like(malla, dtype=bool)
[0m    [1m^[0m[0m


In [17]:
def busca_min_np_jit(malla):
    minimos = np.zeros_like(malla, dtype=bool)
    _busca_min(malla, minimos)
    return np.nonzero(minimos)
@numba.jit(nopython=True)
def _busca_min(malla, minimos):
    for i in range(1, malla.shape[1]-1):
        for j in range(1, malla.shape[0]-1):
            if (malla[j, i] < malla[j-1, i-1] and
                malla[j, i] < malla[j-1, i] and
                malla[j, i] < malla[j-1, i+1] and
                malla[j, i] < malla[j, i-1] and
                malla[j, i] < malla[j, i+1] and
                malla[j, i] < malla[j+1, i-1] and
                malla[j, i] < malla[j+1, i] and
                malla[j, i] < malla[j+1, i+1]):
                minimos[i, j] = True

In [18]:
busca_min_np_jit(data)

(array([   1,    1,    1, ..., 1998, 1998, 1998], dtype=int64),
 array([   1,    3,   11, ..., 1968, 1977, 1985], dtype=int64))

In [19]:
np.testing.assert_array_equal(busca_min(data)[0], busca_min_np_jit(data)[0])
np.testing.assert_array_equal(busca_min(data)[1], busca_min_np_jit(data)[1])

In [20]:
%timeit busca_min_np_jit(data)

78.6 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
